In [1]:
import org.apache.spark.sql.types.{DoubleType, IntegerType, LongType, TimestampType,  StringType, StructField, StructType}

import  org.apache.spark.sql.functions._

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.80.128:4043
SparkContext available as 'sc' (version = 3.1.3, master = local[*], app id = local-1648407801753)
SparkSession available as 'spark'


import org.apache.spark.sql.types.{DoubleType, IntegerType, LongType, TimestampType, StringType, StructField, StructType}
import org.apache.spark.sql.functions._


In [2]:
var intraDayMinParquetDf = spark.read
    .format("parquet")
    .option("timestampFormat", "yyyyMMdd") //20200803
    .load("hdfs://localhost:9000/silver/")

intraDayMinParquetDf: org.apache.spark.sql.DataFrame = [Symbol: string, Date: string ... 7 more fields]


In [3]:
intraDayMinParquetDf.count()

res0: Long = 27812025


In [4]:
intraDayMinParquetDf = intraDayMinParquetDf.withColumn("DateTimeStr", concat( col("Date"), lit(" "), col("Time")))
    .withColumn("DateTime", to_timestamp(col("DateTimeStr"), "yyyyMMdd hh:mm" ))
    .withColumn("Date", to_timestamp(col("Date"), "yyyyMMdd" ))
    .drop($"DateTimeStr")
    .drop($"Time")
    .drop($"Date")

intraDayMinParquetDf: org.apache.spark.sql.DataFrame = [Symbol: string, Open: double ... 6 more fields]


In [5]:
intraDayMinParquetDf.withColumn("Year", date_format(col("DateTime"), "yyyy"))
              .withColumn("Month", date_format(col("DateTime"), "MM"))
              .withColumn("Day", date_format(col("DateTime"), "dd"))
                .write
                .partitionBy("Year", "Month", "Day")
                .format("parquet")
                .mode("overwrite")
                .save("hdfs://localhost:9000/intraday/")